In [14]:
from prelu_cnn import CNN

from datasets import load_from_disk

# Load training dataset
dataset_path = "../processed_datasets/imagenet_processor"
ds = load_from_disk(dataset_path)
ds = ds["train"].select(range(1000))



Loading dataset from disk:   0%|          | 0/1550 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/61 [00:00<?, ?it/s]

Loading dataset from disk:   0%|          | 0/121 [00:00<?, ?it/s]

In [17]:
import torch
ds['pixel_values'].shape


AttributeError: 'list' object has no attribute 'shape'

In [9]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Move model and inputs to GPU if available
model = CNN(use_prelu=False, use_builtin_conv=True).to(device)
inputs = inputs.to(device)
labels = labels.to(device)

outputs = model(inputs)
outputs

Using device: cuda


tensor([[-0.2563,  0.1757, -0.0993,  ...,  0.3784,  0.3742,  0.2112],
        [-0.8987, -0.0303, -0.4067,  ...,  0.2573,  0.4382, -0.4506],
        [-0.1128,  0.4342,  0.3110,  ...,  0.3230, -0.5133,  0.0757],
        ...,
        [-0.0065,  0.1571,  0.1113,  ...,  0.2351, -0.0136,  0.0640],
        [-0.3416, -0.0612,  0.1698,  ...,  0.5790, -0.3259,  0.1333],
        [-0.4196,  0.5339,  0.1695,  ..., -0.1089,  0.4114, -0.2792]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [10]:
import numpy as np

# axis=1 means we're finding the maximum value along the second dimension (columns)
# For a 2D tensor with shape (batch_size, num_classes), axis=1 gives us the class index
# with the highest probability for each sample in the batch
predictions = np.argmax(outputs.detach().cpu().numpy(), axis=1)
print("Predictions:", predictions)
print("Labels:", labels.cpu().numpy())
print("Correct predictions:", (predictions == labels.cpu().numpy()).sum(), "out of", len(predictions))


Predictions: [789 200 419 673 840 924 252 175 778 342 252 419 241 533 530 518 911 924
 427  92 480 146 185 204 560  20 276 846 336 505 557 172 419 241 812 533
 992 525 150 968 611 862 617 659 322 505 175 156  99 967 667 256 705 562
 533 705 241 518 176 190 772 846 655 667  76 524 175 326 241 928 993 342
 715 629 517 789 190 725 336  74 212 256 241  94 804 745 667 190 252 256
 388 988 992 667  52 277 252 342 705 934 525 287 845 600 144  16 369 756
 992 369 144 369 190  47 972 419 494 277 263 276 241  16 811 988 793 508
 463 146]
Labels: [726 917  13 939   6 983 655 579 702 845  69 822 575 906 752 219 192 191
 292 848 108 372 765 473 525 639 686  99 127  76 905 550  30 634 907 979
 718 154 914 293   9 922 130  33 968 719 653 840 139 198 236 304 547 940
 215 853 805  28 104  67 311 429 941 950 603 971 486 504 497 670 459 559
 940   9 829 888 773 784 782 579 714 274 146 245 761 256 326 264 827 690
 973 130  91 615 301 361 614 572  92 303 304 799 362 222 371 547 735 121
 165 509  79 556 821

In [11]:
import torch.nn as nn

loss_fct = nn.CrossEntropyLoss()
loss = loss_fct(outputs, labels)
loss

tensor(6.9573, device='cuda:0', grad_fn=<NllLossBackward0>)